In [ ]:
import itertools
import numpy
import pulp

In [ ]:
lights = []
buttons = []
joltages = []

f = open("input.txt")

for line in f:
    split = line.split()
    lights.append([char == '#' for char in split[0][1:-1]])
    this_buttons = []
    for button_str in split[1:-1]:
        this_buttons.append([int(char) for char in button_str[1:-1].split(',')])
    buttons.append(this_buttons)
    joltages.append([int(char) for char in split[-1][1:-1].split(',')])

f.close()

print(lights[0])
print(buttons[0])
print(joltages[0])

Part 1

In [ ]:
def light_config( n_lights, buttons, presses ):
    light = [False for i in range(0, n_lights)]

    for press in presses:
        for idx in buttons[press]:
            light[idx] = not light[idx]

    return light

light_config(3, [[0, 1], [0, 2]], [1, 0])

In [ ]:
total = 0

for i in range(0, len(lights)):
    done_this_row = False
    for num_pushes in range(0, len(lights[i])):
        for comb in itertools.combinations(range(0, len(buttons[i])), num_pushes):
            if light_config( len(lights[i]), buttons[i], comb ) == lights[i]:
                total += num_pushes
                done_this_row = True
                break
        if done_this_row:
            break

print(total)

Part 2

In [ ]:
def matrix_of_buttons( n_joltage, buttons ):
    M = []
    for i in range(0, n_joltage):
        this_row = []
        for j in range(0, len(buttons)):
            this_row.append(1 if i in buttons[j] else 0)
        M.append(this_row)

    return M


In [ ]:
total = 0

for machine in range(0, len(joltages)):
    A = numpy.array(matrix_of_buttons(len(joltages[machine]), buttons[machine]))
    b = numpy.array(joltages[machine])
    #print(A)
    #print(b)

    problem = pulp.LpProblem("prob", pulp.LpMinimize)

    #Constraints
    x = numpy.empty(A[0].shape[0], dtype=object)
    for i in range(A[0].shape[0]):
        x[i] = pulp.LpVariable("x" + str(i), lowBound=0, upBound=None, cat='Integer')

    for row in range(A.shape[0]):
        problem += numpy.dot(A[row], x) == b[row]

    #Objective
    problem += numpy.sum(x)

    status = problem.solve()
    print(pulp.LpStatus[status])
    this_total = pulp.value(sum(x))
    #print(this_total)
    total += this_total

print(total)